In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Swing Trading Strategy (EMA/RSI/MACD/Bollinger/ATR)
with brokerage fees, volume-spike toggle, robust/logged grid search,
and proper capital allocation when ALLOW_MULTIPLE_POSITIONS=True.

- Entry (all on signal day; filled next day's open):
  • Up-trend: EMA_fast > EMA_slow
  • RSI oversold cross up: RSI_y < OS and RSI_t >= OS
  • MACD bullish crossover: MACD line crosses above its signal
  • Bollinger reversion: Close_{t-1} < Lower_{t-1} AND Close_t > Lower_t
  • Volume spike (optional): Vol_t >= VOL_MULT * SMA(VOL_SMA)

- Exit (any):
  • RSI > overbought
  • MACD bearish cross
  • Upper-band reversal: touched upper yesterday; today close < upper
  • Intraday SL/TP using High/Low (if both hit, assume SL first conservatively)

Outputs:
  - trades.csv, equity.csv
  - grid_results.csv (+ grid_results_partial.csv during runs)
"""

from __future__ import annotations
import os, sys, math, json, time, itertools, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import multiprocessing as mp
from multiprocessing.dummy import Pool as ThreadPool  # threads backend

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# CONFIG
# =========================
CONFIG = {
    # Universe (use Yahoo symbols; add .NS for NSE)
    "TICKERS": ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS'],


    "START_DATE": "2010-01-01",
    "END_DATE": None,                 # None -> today (IST-normalized dates)

    # Cache
    "CACHE_DIR": "./cache_yf",
    "FORCE_REFRESH": False,

    # Strategy params (defaults for base run; grid can override)
    "EMA_FAST": 20,
    "EMA_SLOW": 50,
    "RSI_LEN": 14,
    "RSI_OVERSOLD": 30.0,
    "RSI_OVERBOUGHT": 70.0,
    "MACD_FAST": 12,
    "MACD_SLOW": 26,
    "MACD_SIG": 9,
    "BB_LEN": 20,
    "BB_STD": 2.0,
    "VOL_SMA": 20,
    "VOL_MULT": 1.5,
    "USE_VOLUME_SPIKE": False,        # <<< toggle the volume gate in ENTRY

    # ATR/Stops/Targets
    "ATR_LEN": 14,
    "SL_ATR_MULT": 1.5,
    "TARGET_R": 2.0,

    # Sizing
    "RISK_PER_TRADE_PCT": 0.01,      # 1% of equity risked; set 0 to use FIXED_SHARES
    "FIXED_SHARES": 0,

    # Fees/slippage
    "APPLY_FEES": True,
    "SLIPPAGE_BPS": 0,               # per fill (basis points)

    # Book
    "START_CAPITAL_INR": 200_000.0,
    "ALLOW_MULTIPLE_POSITIONS": True,

    # Allocation when multiple positions allowed:
    # "split" = split start capital equally across tickers
    # "full"  = simulate each ticker with full capital (optimistic; rebase later)
    "CAPITAL_ALLOCATION": "split",

    # --- Grid Search ---
    "RUN_GRID": False,
    "GRID_PROCESSES": 4,
    "GRID_BACKEND": "auto",          # auto -> threads in notebook, processes in .py
    "GRID_LOG_EVERY": 1,             # log every N results
    "GRID_PARTIAL_SAVE_EVERY": 25,   # write partial CSV every N results (0 to disable)
    "GRID_SCORE_PRIMARY": "sharpe",  # (sharpe|sortino|cagr_pct|profit_factor)
    "GRID_SCORE_SECONDARY": "profit_factor",
    "GRID_SCORE_TERTIARY": "cagr_pct",
    "GRID_MAX_COMBOS": 5000,         # 0/None to disable random capping
    "GRID_RANDOM_SEED": 42,

    # Parameter grid (includes MACD & BB; modest ranges to avoid explosion)
    "GRID": {
        "USE_VOLUME_SPIKE": [False],
        "VOL_MULT": [1.2, 1.5, 2.0],

        "RSI_OVERSOLD": [25.0, 30.0, 35.0],
        "RSI_OVERBOUGHT": [65.0, 70.0, 75.0],

        "EMA_FAST": [10, 20],
        "EMA_SLOW": [50, 100],

        # MACD & BB sweeps (small/sane)
        "MACD_FAST": [10, 12],     # keep < MACD_SLOW
        "MACD_SLOW": [26],         # classic slow
        "MACD_SIG":  [9, 12],

        "BB_LEN": [20],
        "BB_STD": [1.5, 2.0],

        "SL_ATR_MULT": [1.0, 1.5, 2.0],
        "TARGET_R":   [1.5, 2.0, 2.5],
    },
}

if CONFIG["END_DATE"] is None:
    CONFIG["END_DATE"] = datetime.now().date().isoformat()

# =========================
# Logging helpers
# =========================
def setup_logging(level: str = "INFO"):
    lvl = getattr(logging, level.upper(), logging.INFO)
    logging.basicConfig(
        level=lvl,
        format="%(asctime)s | %(levelname)s | %(message)s",
        datefmt="%H:%M:%S",
    )

def in_notebook() -> bool:
    return "ipykernel" in sys.modules or "IPython" in sys.modules

# =========================
# Fees (your function)
# =========================
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not CONFIG["APPLY_FEES"]: return 0.0
    BROKER_PCT = 0.001; BROKER_MIN = 5.0; BROKER_CAP = 20.0
    STT_PCT = 0.001; STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297; SEBI_PCT = 0.000001; IPFT_PCT = 0.000001
    GST_PCT = 0.18; DP_SELL = 20.0 if turnover_sell >= 100 else 0.0
    def _broker(turn): return max(BROKER_MIN, min(turn*BROKER_PCT, BROKER_CAP)) if turn>0 else 0
    brb, brs = _broker(turnover_buy), _broker(turnover_sell)
    stt = STT_PCT*(turnover_buy+turnover_sell)
    stamp = STAMP_BUY_PCT*turnover_buy
    exch = EXCH_PCT*(turnover_buy+turnover_sell)
    sebi = SEBI_PCT*(turnover_buy+turnover_sell)
    ipft = IPFT_PCT*(turnover_buy+turnover_sell)
    dp = DP_SELL
    gst = GST_PCT*(brb+brs+dp+exch+sebi+ipft)
    return float((brb+brs)+stt+stamp+exch+sebi+ipft+dp+gst)

# =========================
# Data utils
# =========================
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_ist_date_index(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df.index, pd.DatetimeIndex): return df
    df = df.copy()
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert("Asia/Kolkata")
    else:
        df.index = df.index.tz_convert("Asia/Kolkata")
    df.index = pd.to_datetime(df.index.date)
    return df

def load_yf(ticker: str, start: str, end: str, cache_dir: str, force_refresh: bool) -> pd.DataFrame:
    """Download/cached Yahoo data; always keep OHLCV; synthesize Adj Close when missing."""
    ensure_dir(cache_dir)
    safe = ticker.replace("^", "_")
    path = os.path.join(cache_dir, f"{safe}.csv")

    def _clean(df0: pd.DataFrame) -> pd.DataFrame:
        if df0.empty: return df0
        need = ["Open","High","Low","Close","Adj Close","Volume"]
        have = [c for c in need if c in df0.columns]
        df = df0[have].copy()
        if "Adj Close" not in df and "Close" in df:
            df["Adj Close"] = df["Close"]
        df = to_ist_date_index(df)
        df = df[~df.index.duplicated(keep="first")].sort_index()
        return df

    if (not force_refresh) and os.path.exists(path):
        df = pd.read_csv(path, parse_dates=["Date"]).set_index("Date")
        df = _clean(df)
        if not df.empty:
            return df

    df = yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        raise ValueError(f"No data for {ticker}")
    df = _clean(df)
    df.to_csv(path, index_label="Date")
    return df

# =========================
# Indicators
# =========================
def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False).mean()

def rsi_wilder(close: pd.Series, length: int = 14) -> pd.Series:
    delta = close.diff()
    up = delta.clip(lower=0.0)
    down = -delta.clip(upper=0.0)
    roll_up = up.ewm(alpha=1/length, adjust=False).mean()
    roll_down = down.ewm(alpha=1/length, adjust=False).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    return (100 - (100 / (1 + rs))).fillna(50.0)

def macd(close: pd.Series, fast=12, slow=26, sig=9) -> Tuple[pd.Series,pd.Series,pd.Series]:
    ema_f = ema(close, fast)
    ema_s = ema(close, slow)
    line = ema_f - ema_s
    signal = ema(line, sig)
    hist = line - signal
    return line, signal, hist

def bollinger(close: pd.Series, length=20, std=2.0) -> Tuple[pd.Series,pd.Series,pd.Series]:
    ma = close.rolling(length).mean()
    sd = close.rolling(length).std(ddof=0)
    upper = ma + std * sd
    lower = ma - std * sd
    return ma, upper, lower

def atr_wilder(high: pd.Series, low: pd.Series, close: pd.Series, length=14) -> pd.Series:
    prev_close = close.shift(1)
    tr = pd.concat([(high - low), (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)
    return tr.ewm(alpha=1/length, adjust=False).mean()

# =========================
# Signals
# =========================
def build_signals(df: pd.DataFrame, params: Dict) -> pd.DataFrame:
    o,h,l,c,v = df["Open"], df["High"], df["Low"], df["Close"], df["Volume"]

    ema_fast = ema(c, params["EMA_FAST"])
    ema_slow = ema(c, params["EMA_SLOW"])
    uptrend = ema_fast > ema_slow

    rsi = rsi_wilder(c, params["RSI_LEN"])
    rsi_oversold_cross = (rsi.shift(1) < params["RSI_OVERSOLD"]) & (rsi >= params["RSI_OVERSOLD"])

    macd_line, macd_sig, _ = macd(c, params["MACD_FAST"], params["MACD_SLOW"], params["MACD_SIG"])
    macd_bull_cross = ((macd_line.shift(1) - macd_sig.shift(1)) <= 0) & ((macd_line - macd_sig) > 0)

    bb_ma, bb_up, bb_lo = bollinger(c, params["BB_LEN"], params["BB_STD"])
    bb_revert = (c.shift(1) < bb_lo.shift(1)) & (c > bb_lo)

    vol_sma = v.rolling(params["VOL_SMA"]).mean()
    vol_spike = v >= (params["VOL_MULT"] * vol_sma)

    atr = atr_wilder(h, l, c, params["ATR_LEN"])

    # ENTRY (volume optional via toggle)
    if params.get("USE_VOLUME_SPIKE", True):
        entry = uptrend & rsi_oversold_cross & macd_bull_cross & bb_revert & vol_spike
    else:
        entry = uptrend & rsi_oversold_cross & macd_bull_cross & bb_revert

    # Exits (close-based)
    rsi_overbought = rsi > params["RSI_OVERBOUGHT"]
    macd_bear = ((macd_line.shift(1) - macd_sig.shift(1)) >= 0) & ((macd_line - macd_sig) < 0)
    bb_upper_rev = (c.shift(1) >= bb_up.shift(1)) & (c < bb_up)

    out = pd.DataFrame({
        "Open": o, "High": h, "Low": l, "Close": c, "Volume": v,
        "atr": atr,
        "entry": entry.astype(int),
        "exit_rsi_ob": rsi_overbought.astype(int),
        "exit_macd_bear": macd_bear.astype(int),
        "exit_bb_upper_rev": bb_upper_rev.astype(int),
        # diagnostics
        "entry_no_vol": (uptrend & rsi_oversold_cross & macd_bull_cross & bb_revert).astype(int),
        "vol_spike": vol_spike.astype(int),
    }, index=df.index)
    return out

# =========================
# Backtest
# =========================
@dataclass
class Position:
    ticker: str
    entry_date: pd.Timestamp
    entry_price: float
    shares: int
    sl: float
    tp: float

def _apply_slippage(price: float, bps: int) -> float:
    return price * (1 + bps/10000.0) if bps > 0 else price

def simulate_ticker(ticker: str, sig: pd.DataFrame, params: Dict, start_capital: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
    sl_bps = CONFIG["SLIPPAGE_BPS"]
    risk_pct = CONFIG["RISK_PER_TRADE_PCT"]
    fixed_sh = CONFIG["FIXED_SHARES"]

    cash = start_capital
    pos: Optional[Position] = None
    dates = sig.index
    eq = pd.Series(index=dates, dtype=float)
    trades = []

    for i, d in enumerate(dates):
        row = sig.loc[d]
        h,l,c = row["High"], row["Low"], row["Close"]

        # Exits first - intraday SL/TP precedence
        if pos is not None:
            sl_hit = l <= pos.sl
            tp_hit = h >= pos.tp
            exit_reason = None
            exit_price = None

            if sl_hit and tp_hit:
                exit_reason = "exit_stop_intraday_both"  # conservative
                exit_price = pos.sl
            elif sl_hit:
                exit_reason = "exit_stop"
                exit_price = pos.sl
            elif tp_hit:
                exit_reason = "exit_target"
                exit_price = pos.tp
            elif row["exit_rsi_ob"] or row["exit_macd_bear"] or row["exit_bb_upper_rev"]:
                exit_reason = "exit_signal"
                exit_price = c

            if exit_reason is not None:
                px = _apply_slippage(exit_price, sl_bps)
                proceeds = px * pos.shares
                fees = calc_fees(pos.entry_price * pos.shares, proceeds)
                pnl = proceeds - fees - (pos.entry_price * pos.shares)
                cash += proceeds - fees
                trades.append({
                    "ticker": pos.ticker, "side": "SELL", "date": d,
                    "price": px, "shares": pos.shares, "reason": exit_reason, "pnl": pnl
                })
                pos = None

        # Entry for next day's open
        if row["entry"] == 1 and pos is None and (i + 1 < len(dates)):
            next_d = dates[i+1]
            next_open = sig.loc[next_d, "Open"]
            atr = sig.loc[d, "atr"]
            risk_per_share = max(1e-9, params["SL_ATR_MULT"] * atr)

            if fixed_sh > 0:
                size = int(fixed_sh)
            elif risk_pct > 0:
                risk_cash = cash * risk_pct
                size = int(max(0, math.floor(risk_cash / risk_per_share)))
            else:
                size = int(max(0, cash // max(1.0, next_open)))

            if size > 0 and next_open > 0:
                buy_px = _apply_slippage(next_open, sl_bps)
                cost = buy_px * size
                sl = buy_px - params["SL_ATR_MULT"] * atr
                tp = buy_px + params["TARGET_R"] * (params["SL_ATR_MULT"] * atr)
                fees = calc_fees(cost, 0.0)
                total_cost = cost + fees
                if total_cost <= cash:
                    cash -= total_cost
                    pos = Position(ticker=ticker, entry_date=next_d, entry_price=buy_px, shares=size, sl=sl, tp=tp)
                    trades.append({
                        "ticker": ticker, "side": "BUY", "date": next_d,
                        "price": buy_px, "shares": size, "reason": "entry_rule", "pnl": 0.0
                    })

        # Mark to market
        mtm = cash
        if pos is not None:
            mtm += c * pos.shares
        eq.loc[d] = mtm

    # Force-close at the end
    if pos is not None:
        d = dates[-1]
        c = sig.loc[d, "Close"]
        px = _apply_slippage(c, sl_bps)
        proceeds = px * pos.shares
        fees = calc_fees(pos.entry_price * pos.shares, proceeds)
        pnl = proceeds - fees - (pos.entry_price * pos.shares)
        cash += proceeds - fees
        trades.append({
            "ticker": pos.ticker, "side": "SELL", "date": d,
            "price": px, "shares": pos.shares, "reason": "eod_force_close", "pnl": pnl
        })
        eq.iloc[-1] = cash

    trades_df = pd.DataFrame(trades, columns=["ticker","side","date","price","shares","reason","pnl"])
    return trades_df, eq.to_frame(name=ticker)

# =========================
# Metrics
# =========================
def _metrics_from_equity(eq: pd.Series) -> Dict:
    if eq.empty:
        return dict(sharpe=0.0, sortino=0.0, max_drawdown_pct=0.0, cagr_pct=0.0)
    rets = eq.pct_change().fillna(0.0)
    mean = rets.mean(); std = rets.std(ddof=0)
    neg = rets[rets<0].std(ddof=0)
    sharpe = 0.0 if std == 0 else (mean/std) * np.sqrt(252)
    sortino = 0.0 if (neg == 0 or np.isnan(neg)) else (mean/neg) * np.sqrt(252)
    dd = (eq / eq.cummax() - 1.0).min()
    days = (eq.index[-1] - eq.index[0]).days
    cagr = 0.0 if days <= 0 else (eq.iloc[-1]/eq.iloc[0])**(365.25/days) - 1.0
    return dict(sharpe=float(sharpe), sortino=float(sortino),
                max_drawdown_pct=100*float(dd), cagr_pct=100*float(cagr))

def evaluate(trades_all: pd.DataFrame, equity_all: pd.DataFrame) -> Dict:
    eq = equity_all["equity"] if ("equity" in equity_all.columns) else pd.Series(dtype=float)
    metrics = _metrics_from_equity(eq)

    closed = trades_all[trades_all["side"]=="SELL"]
    n = int(len(closed))
    win_rate = 100.0 * (closed["pnl"] > 0).sum() / n if n > 0 else 0.0
    avg_pnl = float(closed["pnl"].mean()) if n > 0 else 0.0
    gp = closed["pnl"].clip(lower=0).sum()
    gl = -closed["pnl"].clip(upper=0).sum()
    profit_factor = float(gp/gl) if gl > 0 else (float("inf") if gp > 0 else 0.0)

    metrics.update(dict(
        trades=n,
        win_rate_pct=float(win_rate),
        avg_pnl_inr=float(avg_pnl),
        profit_factor=float(profit_factor),
        final_equity_inr=float(eq.iloc[-1]) if not eq.empty else CONFIG["START_CAPITAL_INR"]
    ))
    return metrics

# =========================
# Run once over all tickers (with capital allocation fix)
# =========================
def run_once(params: Dict) -> Tuple[pd.DataFrame, pd.DataFrame, Dict]:
    tickers = list(CONFIG["TICKERS"])
    start, end = CONFIG["START_DATE"], CONFIG["END_DATE"]
    cache, force = CONFIG["CACHE_DIR"], CONFIG["FORCE_REFRESH"]

    # Load data & build signals
    data_map = {}
    for tkr in tickers:
        try:
            data_map[tkr] = load_yf(tkr, start, end, cache, force)
        except Exception as e:
            logging.warning(f"[DATA] {tkr}: {e}")

    if not data_map:
        raise RuntimeError("No data loaded for any ticker.")

    sig_map = {tkr: build_signals(df, params) for tkr, df in data_map.items()}

    # ---- Capital allocation fix ----
    n_tickers = len(sig_map)
    if CONFIG["ALLOW_MULTIPLE_POSITIONS"]:
        if CONFIG.get("CAPITAL_ALLOCATION", "split") == "full":
            per_start = CONFIG["START_CAPITAL_INR"]     # optimistic; we'll rebase later
        else:
            per_start = CONFIG["START_CAPITAL_INR"] / max(1, n_tickers)  # split equally
    else:
        per_start = CONFIG["START_CAPITAL_INR"]

    # Simulate each ticker independently; sum MTM and rebase to start capital
    trades_list, eq_list = [], []
    for tkr, sig in sig_map.items():
        tdf, eq = simulate_ticker(tkr, sig, params, start_capital=per_start)
        trades_list.append(tdf); eq_list.append(eq)

    trades_all = pd.concat(trades_list, ignore_index=True) if trades_list else pd.DataFrame(
        columns=["ticker","side","date","price","shares","reason","pnl"]
    )

    if eq_list:
        eq_all = pd.concat(eq_list, axis=1).fillna(method="ffill").fillna(0.0)
        # Rebase summed equity to START_CAPITAL_INR to avoid double counting book capital
        first_val = float(eq_all.sum(axis=1).iloc[0])
        if first_val == 0:
            equity = pd.Series(CONFIG["START_CAPITAL_INR"], index=eq_all.index, name="equity")
        else:
            scale = CONFIG["START_CAPITAL_INR"]/first_val
            equity = (eq_all.sum(axis=1) * scale).rename("equity")
        equity_df = equity.to_frame()
    else:
        equity_df = pd.DataFrame(columns=["equity"])

    metrics = evaluate(trades_all, equity_df)
    return trades_all, equity_df, metrics

# =========================
# Grid helpers
# =========================
def _grid_run_combo(args):
    """Top-level worker usable by threads/processes (picklable)."""
    vals, base, keys, run_once_fn = args
    params = base.copy()
    for k, v in zip(keys, vals):
        params[k] = v
    try:
        _, _, m = run_once_fn(params)
        return {"params": json.dumps(params), **m}
    except Exception as e:
        return {"params": json.dumps(params), "error": str(e)}

def _score_row(row: dict) -> tuple:
    def getf(k):
        try:
            return float(row.get(k, float("nan")))
        except Exception:
            return float("nan")
    pri = getf(CONFIG.get("GRID_SCORE_PRIMARY", "sharpe"))
    sec = getf(CONFIG.get("GRID_SCORE_SECONDARY", "profit_factor"))
    ter = getf(CONFIG.get("GRID_SCORE_TERTIARY", "cagr_pct"))
    pri = pri if np.isfinite(pri) else -1e9
    sec = sec if np.isfinite(sec) else -1e9
    ter = ter if np.isfinite(ter) else -1e9
    return (pri, sec, ter)

def _short_params(p_json: str) -> str:
    try:
        p = json.loads(p_json)
        keys = ["USE_VOLUME_SPIKE","VOL_MULT","EMA_FAST","EMA_SLOW",
                "MACD_FAST","MACD_SLOW","MACD_SIG",
                "BB_LEN","BB_STD",
                "RSI_OVERSOLD","RSI_OVERBOUGHT",
                "SL_ATR_MULT","TARGET_R"]
        return "{" + ", ".join(f"{k}={p[k]}" for k in keys if k in p) + "}"
    except Exception:
        return p_json

# =========================
# Grid search (logged, constrained, optional capping)
# =========================
def grid_search() -> pd.DataFrame:
    base = dict(
        EMA_FAST=CONFIG["EMA_FAST"], EMA_SLOW=CONFIG["EMA_SLOW"],
        RSI_LEN=CONFIG["RSI_LEN"], RSI_OVERSOLD=CONFIG["RSI_OVERSOLD"], RSI_OVERBOUGHT=CONFIG["RSI_OVERBOUGHT"],
        MACD_FAST=CONFIG["MACD_FAST"], MACD_SLOW=CONFIG["MACD_SLOW"], MACD_SIG=CONFIG["MACD_SIG"],
        BB_LEN=CONFIG["BB_LEN"], BB_STD=CONFIG["BB_STD"],
        VOL_SMA=CONFIG["VOL_SMA"], VOL_MULT=CONFIG["VOL_MULT"], USE_VOLUME_SPIKE=CONFIG["USE_VOLUME_SPIKE"],
        ATR_LEN=CONFIG["ATR_LEN"], SL_ATR_MULT=CONFIG["SL_ATR_MULT"], TARGET_R=CONFIG["TARGET_R"]
    )

    grid = CONFIG["GRID"]
    keys = list(grid.keys())
    raw = list(itertools.product(*[grid[k] for k in keys]))

    # ---- constraint filter (avoid silly combos) ----
    filtered = []
    for vals in raw:
        d = {k: v for k, v in zip(keys, vals)}
        # EMA fast < slow
        ema_fast = d.get("EMA_FAST", base["EMA_FAST"])
        ema_slow = d.get("EMA_SLOW", base["EMA_SLOW"])
        if ema_fast >= ema_slow:
            continue
        # MACD fast < slow (if provided)
        macd_fast = d.get("MACD_FAST", base["MACD_FAST"])
        macd_slow = d.get("MACD_SLOW", base["MACD_SLOW"])
        if macd_fast >= macd_slow:
            continue
        # BB std must be positive
        bb_std = d.get("BB_STD", base["BB_STD"])
        if float(bb_std) <= 0:
            continue
        filtered.append(vals)

    # ---- optional random subsample cap ----
    maxc = int(CONFIG.get("GRID_MAX_COMBOS") or 0)
    if maxc and len(filtered) > maxc:
        rng = np.random.default_rng(CONFIG.get("GRID_RANDOM_SEED", 42))
        idx = rng.choice(len(filtered), size=maxc, replace=False)
        filtered = [filtered[i] for i in idx]

    args_list = [(vals, base, keys, run_once) for vals in filtered]
    total = len(args_list)
    logging.info(f"[GRID] prepared {total} combos (from {len(raw)} raw) after constraints/cap")

    # Backend select
    backend = CONFIG.get("GRID_BACKEND", "auto")
    if backend == "auto":
        backend = "thread" if in_notebook() else "process"
    nworkers = max(1, min(CONFIG["GRID_PROCESSES"], os.cpu_count() or 1))
    logging.info(f"[GRID] backend={backend} | workers={nworkers}")

    rows = []
    best = None
    best_score = (-1e9, -1e9, -1e9)
    t0 = time.time()

    def handle_row(i: int, row: dict):
        nonlocal best, best_score, rows
        rows.append(row)
        if "error" in row:
            logging.warning(f"[{i}/{total}] error: {row['error']} | params={_short_params(row['params'])}")
        else:
            score = _score_row(row)
            is_best = score > best_score
            if is_best:
                best, best_score = row, score
            # progress log
            log_every = max(1, int(CONFIG.get("GRID_LOG_EVERY", 1)))
            if (i % log_every == 0) or is_best:
                logging.info(
                    f"[{i}/{total}] sh={row.get('sharpe')} pf={row.get('profit_factor')} "
                    f"cagr={row.get('cagr_pct')}% maxDD={row.get('max_drawdown_pct')}% "
                    f"| best_by_{CONFIG.get('GRID_SCORE_PRIMARY','sharpe')} "
                    f"sh={best.get('sharpe') if best else None}, "
                    f"pf={best.get('profit_factor') if best else None}, "
                    f"cagr={best.get('cagr_pct') if best else None}% "
                    f"| params={_short_params(row['params'])}"
                )
        # partial save
        save_every = int(CONFIG.get("GRID_PARTIAL_SAVE_EVERY", 0) or 0)
        if save_every and (i % save_every == 0):
            pd.DataFrame(rows).to_csv("grid_results_partial.csv", index=False)

    # Execute
    if nworkers == 1 or backend == "thread":
        iterator = map(_grid_run_combo, args_list)
        if nworkers > 1 and backend == "thread":
            with ThreadPool(processes=nworkers) as pool:
                iterator = pool.imap_unordered(_grid_run_combo, args_list)
                for i, row in enumerate(iterator, start=1):
                    handle_row(i, row)
        else:
            for i, row in enumerate(iterator, start=1):
                handle_row(i, row)
    else:
        ctx = mp.get_context("spawn")
        with ctx.Pool(processes=nworkers) as pool:
            for i, row in enumerate(pool.imap_unordered(_grid_run_combo, args_list), start=1):
                handle_row(i, row)

    # Finalize
    df = pd.DataFrame(rows)
    df.to_csv("grid_results.csv", index=False)

    dt = time.time() - t0
    logging.info(f"[GRID] Done {total} combos in {dt:.1f}s")
    if df.empty:
        logging.warning("[GRID] No results. (Check data/rules.)")
        return df

    # Top 10 summary
    df["_score_tuple"] = df.apply(_score_row, axis=1)
    df_sorted = df.sort_values("_score_tuple", ascending=False).drop(columns=["_score_tuple"])
    top = df_sorted.head(10)
    cols = ["params","trades","win_rate_pct","profit_factor","sharpe","sortino",
            "max_drawdown_pct","cagr_pct","final_equity_inr"]
    logging.info("[GRID] Top results:")
    try:
        print(top[cols].to_string(index=False))
    except Exception:
        print(top.to_string(index=False))
    return df_sorted

# =========================
# Main
# =========================
def main():
    setup_logging("INFO")
    print("=== Swing Strategy: EMA/RSI/MACD/Bollinger/ATR ===")
    print(f"Symbols:     {len(CONFIG['TICKERS'])} from CONFIG[TICKERS]")
    print(f"Period:      {CONFIG['START_DATE']} .. {CONFIG['END_DATE']}")
    print(f"Fees:        {'ON' if CONFIG['APPLY_FEES'] else 'OFF'} | Start Capital: ₹{CONFIG['START_CAPITAL_INR']:,}")
    print(f"VolumeGate:  USE_VOLUME_SPIKE={CONFIG['USE_VOLUME_SPIKE']} VOL_MULT={CONFIG['VOL_MULT']}")
    print(f"Multi-Pos:   {CONFIG['ALLOW_MULTIPLE_POSITIONS']} | Allocation: {CONFIG['CAPITAL_ALLOCATION']}")
    print(f"Grid:        {'ON' if CONFIG['RUN_GRID'] else 'OFF'} "
          f"(backend={CONFIG['GRID_BACKEND']}, workers={CONFIG['GRID_PROCESSES']})")

    if CONFIG["RUN_GRID"]:
        df = grid_search()
        if not df.empty:
            best = df.iloc[0]
            print("\n[GRID] BEST:")
            print(f"  score_by={CONFIG['GRID_SCORE_PRIMARY']}, "
                  f"sh={best.get('sharpe')}, pf={best.get('profit_factor')}, "
                  f"cagr={best.get('cagr_pct')}%, maxDD={best.get('max_drawdown_pct')}%")
            print(f"  params={_short_params(best['params'])}")
        return

    # Base run with CONFIG params
    params = dict(
        EMA_FAST=CONFIG["EMA_FAST"], EMA_SLOW=CONFIG["EMA_SLOW"],
        RSI_LEN=CONFIG["RSI_LEN"], RSI_OVERSOLD=CONFIG["RSI_OVERSOLD"], RSI_OVERBOUGHT=CONFIG["RSI_OVERBOUGHT"],
        MACD_FAST=CONFIG["MACD_FAST"], MACD_SLOW=CONFIG["MACD_SLOW"], MACD_SIG=CONFIG["MACD_SIG"],
        BB_LEN=CONFIG["BB_LEN"], BB_STD=CONFIG["BB_STD"],
        VOL_SMA=CONFIG["VOL_SMA"], VOL_MULT=CONFIG["VOL_MULT"], USE_VOLUME_SPIKE=CONFIG["USE_VOLUME_SPIKE"],
        ATR_LEN=CONFIG["ATR_LEN"], SL_ATR_MULT=CONFIG["SL_ATR_MULT"], TARGET_R=CONFIG["TARGET_R"]
    )

    trades, equity, metrics = run_once(params)

    # Save outputs (robust)
    if trades is None or trades.empty:
        logging.info("[INFO] No trades generated.")
        pd.DataFrame(columns=["ticker","side","date","price","shares","reason","pnl"]).to_csv("trades.csv", index=False)
    else:
        if "date" not in trades.columns and isinstance(trades.index, pd.DatetimeIndex):
            trades = trades.reset_index().rename(columns={"index": "date"})
        sort_keys = [c for c in ["date","ticker","side"] if c in trades.columns]
        trades = trades.sort_values(sort_keys) if sort_keys else trades
        trades.to_csv("trades.csv", index=False)

    if equity is None or equity.empty:
        pd.DataFrame(columns=["equity"]).to_csv("equity.csv", index=True)
    else:
        equity.to_csv("equity.csv", index=True)

    print("\n=== Metrics ===")
    print(json.dumps(metrics, indent=2))
    print("\nFiles saved:\n  - trades.csv\n  - equity.csv")

if __name__ == "__main__":
    # Keep this guard for multiprocessing safety on macOS/Windows
    try:
        mp.set_start_method("spawn", force=True)
    except RuntimeError:
        pass
    main()


=== Swing Strategy: EMA/RSI/MACD/Bollinger/ATR ===
Symbols:     500 from CONFIG[TICKERS]
Period:      2010-01-01 .. 2025-10-05
Fees:        ON | Start Capital: ₹200,000.0
VolumeGate:  USE_VOLUME_SPIKE=False VOL_MULT=1.5
Multi-Pos:   True | Allocation: split
Grid:        OFF (backend=auto, workers=4)


18:13:00 | INFO | [INFO] No trades generated.



=== Metrics ===
{
  "sharpe": 3.545084585000063,
  "sortino": 0.0,
  "max_drawdown_pct": 0.0,
  "cagr_pct": 3.6345479311231,
  "trades": 0,
  "win_rate_pct": 0.0,
  "avg_pnl_inr": 0.0,
  "profit_factor": 0.0,
  "final_equity_inr": 350877.1929824561
}

Files saved:
  - trades.csv
  - equity.csv
